<a href="https://colab.research.google.com/github/zacSimo/PysparkAdvanced/blob/main/pyspark_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=3b31fb2847f637a56abade9f2b65c0ec17c43f8949dc7e2d33117f1459042158
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
import pyspark.sql.functions as f

from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Practice').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 13:20:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.sparkContext.setLogLevel("ERROR")

In [5]:
sd_df = spark.read.csv('/kaggle/input/fifa-18-demo-player-dataset/PlayerAttributeData.csv', header=True, inferSchema=True)


In [6]:
sd_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Acceleration: string (nullable = true)
 |-- Aggression: string (nullable = true)
 |-- Agility: string (nullable = true)
 |-- Balance: string (nullable = true)
 |-- Ball control: string (nullable = true)
 |-- Composure: string (nullable = true)
 |-- Crossing: string (nullable = true)
 |-- Curve: string (nullable = true)
 |-- Dribbling: string (nullable = true)
 |-- Finishing: string (nullable = true)
 |-- Free kick accuracy: string (nullable = true)
 |-- GK diving: string (nullable = true)
 |-- GK handling: string (nullable = true)
 |-- GK kicking: string (nullable = true)
 |-- GK positioning: string (nullable = true)
 |-- GK reflexes: string (nullable = true)
 |-- Heading accuracy: string (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Interceptions: string (nullable = true)
 |-- Jumping: string (nullable = true)
 |-- Long passing: string (nullable = true)
 |-- Long shots: string (nullable = true)
 |-- Marking: string (nullable = 

In [7]:
sd_df.count()

17981

#####Apache spark memory

In [8]:
sd_broad_df = f.broadcast(sd_df)

In [9]:
# UDF

def my_func(code: str) -> str:

  return bdData.value.get(code)




In [10]:
from pyspark.sql.types import StringType,ArrayType ,StructType,StructField,IntegerType
spark.udf.register("my_function", my_func, StringType())

<function __main__.my_func(code: str) -> str>

In [11]:
# see how many cores available for spark engine

spark.sparkContext.defaultParallelism

4

In [12]:
# generate ramdom data

import random

randomlist = random.sample(range(1, 40),10)

In [13]:
print(randomlist)

[26, 21, 38, 19, 28, 36, 34, 15, 22, 33]


In [14]:
# parallelize and take it into 4 partitions

rdd1 = spark.sparkContext.parallelize(randomlist, 4)

In [15]:
rdd1

ParallelCollectionRDD[17] at readRDDFromFile at PythonRDD.scala:289

In [16]:
# show exactly the same data as created

rdd1.collect()

[26, 21, 38, 19, 28, 36, 34, 15, 22, 33]

In [17]:
# to see the numbers of partitions

rdd1.getNumPartitions()

4

In [18]:
# to see the data in partitions

rdd1.glom().collect()

[[26, 21], [38, 19], [28, 36], [34, 15, 22, 33]]

In [19]:
rdd1.take(rdd1.count()-1)

[26, 21, 38, 19, 28, 36, 34, 15, 22]

In [20]:
rdd1.glom().collect()[3]

[34, 15, 22, 33]

In [21]:
rdd1.top(3)

[38, 36, 34]

In [22]:
rdd_map = rdd1.map(lambda x: x*2)

rdd_map.collect()

[52, 42, 76, 38, 56, 72, 68, 30, 44, 66]

In [23]:
def myfunc(x):

  if x%2 == 0:

    return x * 2

  else:

    return x*3



rdd_map_2 = rdd1.map(myfunc)

rdd_map_2.collect()

[52, 63, 76, 57, 56, 72, 68, 45, 44, 99]

In [24]:
rdd_filter = rdd1.filter(lambda x: x%2 == 0)

rdd_filter.collect()

[26, 38, 28, 36, 34, 22]

In [25]:
# flatmap collect values into a single list

# used when we want aggregation of the result (reduce)

rdd_flatmap = rdd1.flatMap(lambda x: (x, x*10))

rdd_flatmap.collect()

[26,
 260,
 21,
 210,
 38,
 380,
 19,
 190,
 28,
 280,
 36,
 360,
 34,
 340,
 15,
 150,
 22,
 220,
 33,
 330]

In [26]:
rdd_map_3 = rdd1.map(lambda x: [x, x*10])

rdd_map_3.collect()

[[26, 260],
 [21, 210],
 [38, 380],
 [19, 190],
 [28, 280],
 [36, 360],
 [34, 340],
 [15, 150],
 [22, 220],
 [33, 330]]

In [27]:
rdd_map_3.glom().collect()

[[[26, 260], [21, 210]],
 [[38, 380], [19, 190]],
 [[28, 280], [36, 360]],
 [[34, 340], [15, 150], [22, 220], [33, 330]]]

In [28]:
print(rdd1.max())

print(rdd1.min())

print(rdd1.mean())

print(rdd1.sum())

print(rdd1.stdev())

38
15
27.2
272
7.467261881037788


In [29]:
# mapPartitions() map a function into each partitions

def my_partition_func(iterator):

  sum = 0

  for i in iterator:

    sum += i



  yield sum



rdd_map_partition = rdd1.mapPartitions(my_partition_func)

rdd_map_partition.collect()

[47, 57, 64, 104]

In [30]:
# union

rdd2 = spark.sparkContext.parallelize(random.sample(range(1,20), 10),2)

rdd2.collect()


[9, 15, 6, 18, 3, 1, 16, 12, 17, 7]

In [31]:
rdd1.union(rdd2).collect()

[26, 21, 38, 19, 28, 36, 34, 15, 22, 33, 9, 15, 6, 18, 3, 1, 16, 12, 17, 7]

In [32]:
rdd_union = rdd1.union(rdd2)

rdd_union.glom().collect()

[[26, 21],
 [38, 19],
 [28, 36],
 [34, 15, 22, 33],
 [9, 15, 6, 18, 3],
 [1, 16, 12, 17, 7]]

In [33]:
rdd1.union(rdd2).getNumPartitions()

6

In [34]:
rdd1.union(rdd2).glom().collect()

[[26, 21],
 [38, 19],
 [28, 36],
 [34, 15, 22, 33],
 [9, 15, 6, 18, 3],
 [1, 16, 12, 17, 7]]

In [35]:
rdd_inter = rdd1.intersection(rdd2)

rdd_inter.collect()

[15]

In [36]:
rdd_inter.glom().collect()

[[], [], [], [15], [], []]

In [37]:
rdd_inter.coalesce(1).glom().collect()

[[15]]

In [38]:
# takeSample withreplacement, num, [seed]

rdd1.takeSample(False, 5)

[33, 26, 36, 22, 21]

In [39]:
# takeOrdered

rdd1.takeOrdered(5)

[15, 19, 21, 22, 26]

In [40]:
rdd1.takeOrdered(5, key=lambda x: -x)

[38, 36, 34, 33, 28]

In [41]:
rdd1.reduce(lambda x, y: x+y)

272

In [42]:
rdd_rbk = rdd_union.map(lambda x: (x, x*x)).reduceByKey(lambda x,y: x+y)

In [43]:
rdd_rbk.toDF(schema=["key","value"]).show(vertical=False,truncate=False)

+---+-----+
|key|value|
+---+-----+
|36 |1296 |
|6  |36   |
|18 |324  |
|12 |144  |
|19 |361  |
|1  |1    |
|7  |49   |
|26 |676  |
|38 |1444 |
|21 |441  |
|15 |450  |
|33 |1089 |
|9  |81   |
|3  |9    |
|28 |784  |
|34 |1156 |
|22 |484  |
|16 |256  |
|17 |289  |
+---+-----+



In [44]:
# sortByKey()

rdd_rbk.sortByKey().collect()

[(1, 1),
 (3, 9),
 (6, 36),
 (7, 49),
 (9, 81),
 (12, 144),
 (15, 450),
 (16, 256),
 (17, 289),
 (18, 324),
 (19, 361),
 (21, 441),
 (22, 484),
 (26, 676),
 (28, 784),
 (33, 1089),
 (34, 1156),
 (36, 1296),
 (38, 1444)]

In [45]:
rdd_rbk.countByKey()

defaultdict(int,
            {36: 1,
             6: 1,
             18: 1,
             12: 1,
             19: 1,
             1: 1,
             7: 1,
             26: 1,
             38: 1,
             21: 1,
             15: 1,
             33: 1,
             9: 1,
             3: 1,
             28: 1,
             34: 1,
             22: 1,
             16: 1,
             17: 1})

In [46]:
rdd_gbk = rdd_rbk.groupByKey().collect()

In [47]:
for item in rdd_gbk:

  print(item[0], list(item[1]))

36 [1296]
6 [36]
18 [324]
12 [144]
19 [361]
1 [1]
7 [49]
26 [676]
38 [1444]
21 [441]
15 [450]
33 [1089]
9 [81]
3 [9]
28 [784]
34 [1156]
22 [484]
16 [256]
17 [289]


In [48]:
rdd_gby_1 = rdd_rbk.groupByKey()

In [49]:
import os

# os.environ["PYTHONHASHSEED"] = "false"

In [50]:


rdd_rbk.sortByKey().lookup(10)

[]

In [51]:
rdd_rbk.cache()

PythonRDD[94] at RDD at PythonRDD.scala:53

In [52]:
  rdd_rbk.is_cached

True

In [53]:
rdd_rbk.unpersist()

PythonRDD[94] at RDD at PythonRDD.scala:53

In [54]:
# DF

spark

In [55]:
#df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/kaggle/input/fifa-18-demo-player-dataset/CompleteDataset.csv")

In [56]:
#df.printSchema()

In [57]:
#df.show(truncate=False)

In [58]:
#df.count()

In [59]:
#df.rdd.glom().collect()

In [60]:
#df.rdd.getNumPartitions()

In [61]:
#df2 = df.repartition(4)

In [62]:
#df2.rdd.getNumPartitions()

In [63]:
#df2.rdd.glom().collect()

In [64]:
#df.withColumnRenamed("_c0", "id").show()

In [65]:
#df2 = df.withColumnRenamed("_c0", "id")


In [66]:
#df2.printSchema()

In [67]:
#df2.rdd.getNumPartitions()

In [68]:
#df2 = df2.repartition(6)

In [69]:
#df2.rdd.getNumPartitions()

In [70]:
#df3 = df2.repartition(3, "Nationality")

In [71]:
#df3.rdd.glom().collect()

In [72]:
#df3.rdd.getNumPartitions()

In [73]:
#df3.show()

In [74]:
#df3.na.fill({"CAM": 10, "CB": 1}).show()

In [75]:
#df3.filter(df3["Nationality"] == "Germany").show()

In [76]:
#df3.groupBy("Nationality").count().sort("Nationality").show(50)

In [77]:
#df3.groupBy("Nationality", "Age").count().sort("Nationality","Age").show(50)

In [78]:
from pyspark.sql import *

In [79]:
# visualize with pandas

#df3_res = df3.where(df3["Overall"]>70).filter(df3["Nationality"] == "Albania").groupBy("Nationality", "Age").count().sort("Nationality","Age").toPandas()

#df3_res.plot(kind="bar", x="Nationality", y="count")


In [80]:
#df3_res_2 = df3.where(df3["Overall"]>70).groupBy("Age").count().sort("Age").toPandas()

#df3_res_2.plot(kind="bar", x="Age", y="count")


In [81]:
# UDF

def uppercase_lowercase(str):

  if str:  
      if len(str) > 10:
    
        return str.upper()
    
      else:
    
        return str.lower()
  else:
      return ""



spark.udf.register("uppercase_lowercase", uppercase_lowercase, StringType())

<function __main__.uppercase_lowercase(str)>

In [82]:
#df = spark.read.format("json").load('/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json')

In [83]:
# register df

#df.createOrReplaceTempView("fifa_table")

In [84]:
# df2.select("Name", uppercase_lowercase("Name")).show()

#spark.sql("select comments, uppercase_lowercase(comments) as upper_comments from fifa_table").show()

In [85]:
#broadcast(df)

In [86]:
#df.select("abstract", uppercase_lowercase("abstract")).show()

In [87]:
#df.printSchema()

In [88]:
#df.count()

In [89]:
#df.rdd.getNumPartitions()

In [90]:
#df.rdd.glom().collect()

In [91]:
#df.show()

In [92]:
#df.cache()

In [93]:
spark.sparkContext.defaultParallelism

4

In [94]:
#df.count()

In [95]:
import json

In [96]:
rdd_json = spark.sparkContext.textFile('/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json')

In [97]:
rdd = rdd_json.map(lambda x: json.loads(x))

In [98]:
rdd

PythonRDD[97] at RDD at PythonRDD.scala:53

In [99]:
spark.sparkContext.defaultParallelism

4

In [100]:
# rdd.persist()

In [101]:
rdd.take(2)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

In [102]:
rdd.count()

2011231

In [103]:
rdd.flatMap(lambda x: x.keys()).distinct().collect()

['comments',
 'versions',
 'abstract',
 'report-no',
 'license',
 'update_date',
 'authors',
 'journal-ref',
 'doi',
 'categories',
 'submitter',
 'authors_parsed',
 'id',
 'title']

In [104]:
rdd.map(lambda x: x['license']).distinct().collect()

[None,
 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'http://creativecommons.org/publicdomain/zero/1.0/',
 'http://creativecommons.org/licenses/publicdomain/',
 'http://creativecommons.org/licenses/by-nc-sa/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/3.0/',
 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'http://creativecommons.org/licenses/by-sa/4.0/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/licenses/by/3.0/']

In [105]:
# min title

min_title = rdd.map(lambda x: x['title']).distinct().reduce(lambda x,y: x if (len(x)<len(y)) else y)

print(min_title)

Q


In [106]:
# max title

max_title = rdd.map(lambda x: x['title']).distinct().reduce(lambda x,y: x if (len(x)>len(y)) else y)

print(max_title)

Investigation of the 2-body system with a rotating central body (e. g.
  earth-moon system) within the Projective Unified Field theory: the transfer
  of rotational angular momentum and energy from the central body to the
  orbital 2-body system, the tidal and the non-tidal influences (mechanical,
  general-relativistic Lense-Thirring effect and cosmological
  PUFT-contributions)


In [107]:
import re

In [108]:
# find abbreeviations with 5 or more letters in the "abstract" column

# exept : [^_! /\\:;&#,~<>]

regx = r"\((\w[^_! /\\:;&#,~?${}<>]{5,})\)"

In [109]:
print(re.search(regx, "of the Large Hadron\nCollider (LHCOPS)"))

<re.Match object; span=(29, 37), match='(LHCOPS)'>


In [110]:


rdd.filter(lambda x: re.search(regx, x['abstract'])).count()

188806

In [111]:
# rdd.filter(lambda x: re.search(regx, x['abstract'])).collect()

In [112]:
# Number of archive per month

rdd.map(lambda x: (x['update_date'][5:7], 1)).reduceByKey(lambda x,y: x+y).collect()

[('01', 134247),
 ('06', 191746),
 ('05', 296587),
 ('10', 197755),
 ('11', 297963),
 ('02', 116948),
 ('04', 117126),
 ('03', 126458),
 ('07', 122649),
 ('09', 138978),
 ('08', 138469),
 ('12', 132305)]

In [113]:
from datetime import datetime



def extract_date(line):

  d = datetime.strptime(line, '%Y-%m-%d')

  return d.month



extract_date("2008-12-20")

12

In [114]:
# Number of archive per month

rdd.map(lambda x: (extract_date(x['update_date']), 1)).reduceByKey(lambda x,y: x+y).collect()

[(1, 134247),
 (2, 116948),
 (3, 126458),
 (4, 117126),
 (5, 296587),
 (6, 191746),
 (7, 122649),
 (8, 138469),
 (9, 138978),
 (10, 197755),
 (11, 297963),
 (12, 132305)]

In [115]:
# the average number of pages

# in comments column

# extract only the number be carefull with "None" value

pages = "pages"



def extract_num_pages(comments):

  regex = r"(\d+ pages)"

  r = re.findall(regex, str(comments))

  if r:

   return int(r[0].split(' ')[0])

  return 0



extract_num_pages("Extensively revised version. In the revised version, we have\n  discussed the effect of viscosity on (quark) transverse momentum distribution\n  and on elliptic flow. It was shown that elliptic flow saturates due to\n  non-equilirium correction to the equilibrium distribution function and can\n  not be mimicked by ideal dynamics. 15 pages, 16 figures")

15

In [116]:
rdd_com = rdd.map(lambda x: (x["comments"],extract_num_pages(x["comments"]))).collect()


In [117]:
rdd.map(lambda x: extract_num_pages(x['comments'])).mean()

10.51073496778838

In [118]:
rdd_page_row = rdd.map(lambda x: (x["comments"],extract_num_pages(x["comments"]))).count()

In [119]:
rdd_page_row

2011231

In [120]:
rdd_nn_pages = rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).count()

In [121]:
rdd_nn_pages

2011231

In [122]:
rdd_filter_zero = rdd.filter(lambda x: x != 0).count()

In [123]:
rdd_filter_zero

2011231

In [ ]:
# average with mean function
rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).mean()

In [ ]:
# average with reduce
rdd_reduce = rdd.map(lambda x: extract_num_pages(x["comments"]) if x['comments'] else 0).reduce(lambda x,y: x+y)

In [ ]:
print(rdd_reduce/rdd_filter_zero)

In [ ]:
# using DF
df = spark.read.json("/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json")

In [ ]:
df

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df.printSchema()

In [ ]:
df.count()

In [ ]:
from pyspark.sql.types import *

schema = StructType([
    StructField("authors",StringType(), True),
    StructField("categories",StringType(), True),
    StructField("license",StringType(), True),
    StructField("comments",StringType(), True),
    StructField("abstract",StringType(), True),
    StructField("versions", ArrayType(StringType()), True)
])
schema

In [ ]:
# Binding data to a schama
df = spark.read.json("/kaggle/input/arxiv-metadata/arxiv-metadata-oai-snapshot.json", schema=schema)

In [ ]:
df.show(vertical=True)

In [ ]:
# df count
df.count()

In [ ]:
# Missing values for "comments" and "licence" attributes
# drop
df = df.dropna(subset = ["comments"])

df.count()

In [ ]:
df.show()

In [ ]:
# Missing values for "comments" and "licence" attributes

# replace
df = df.fillna(value = "Unknown", subset = ["license"])
df.count()

In [ ]:
df.show()

In [ ]:
# renamed df to reuse in sql query
df.createOrReplaceTempView("Archive")

query = """select authors, categories from Archive
            where categories like 'math%'
"""
spark.sql(query).show()
print(spark.sql(query).count())

In [ ]:
# Get licences with 5 or more letters in the "abstract"
regexp1 = r"([a-z A-Z]{5,})"

def fiveOrMoreLetters(x):
    if str(x["abstract"]):
        r = re.search(regexp1,x["abstract"])
        if r:
            return x["license"]
        else:
            return None
    else:
        return None

in_js = { 'license': 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
          'abstract': '  Given an abelian algebraic group $A$ over a global field $F$, $\\alpha \\in\nA(F)$.\n'}
fiveOrMoreLetters(in_js)

In [ ]:
df_license = df.withColumn("regxp", f.lit(regexp1)).select(df.license).where(f.regexp("abstract","regxp"))

In [ ]:
df_license.show()

In [ ]:
# 2nd method
query = """select license from Archive
            where abstract REGEXP "([a-z A-Z]{5,})"
"""
spark.sql(query).show()

In [ ]:
# extract the statistics of numbers of pages
spark.udf.register("extract_num_pages", extract_num_pages, IntegerType())
query = """select extract_num_pages(comments) as num_pages from Archive
            where abstract is not null
"""
df_num_pages = spark.sql(query)
df_num_pages.show()
df_num_pages.createOrReplaceTempView("NumPages")
query1 = """select sum(num_pages)/count(*) as avg from NumPages
            where num_pages != 0
"""
spark.sql(query1).show()
query2 = """select AVG(extract_num_pages(comments)) as avg, SUM(extract_num_pages(comments)) as sum,
            STD(extract_num_pages(comments)) as std
            from Archive
            where license = 'Unknown'
"""
spark.sql(query2).show()

In [ ]:
#  'versions': [{'version': 'v1', 'created': 'Sat, 31 Mar 2007 02:26:18 GMT'},
   # {'version': 'v2', 'created': 'Sat, 13 Dec 2008 17:26:00 GMT'}],

# Question 9: Get the average number of pages per day for version 1

# df.select(extract_num_pages(col("comments")) ).where(get_json_object(col("versions")[0], "$.version") == "v1").show()


In [ ]:
# extract day from created field
def extract_day(created_datetime):
    if created_datetime:
        return created_datetime.split(',')[0]
    else:
        return None

extract_day('Thu, 5 Apr 2007 04:48:39 GMT')

In [ ]:
spark.udf.register("extract_day", extract_day, StringType())

In [ ]:

query_avg_pages_per_day = """select AVG(extract_num_pages(comments)) as avg_pages_per_day_v1 , extract_day(get_json_object(versions[0], "$.created")) as day
                             from Archive
                             where get_json_object(versions[0], "$.version") == "v1"
                             group by day
"""
spark.sql(query_avg_pages_per_day).show(truncate=False)

In [ ]:
# Read and load Data to spark
# data source http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

rdd = spark.sparkContext.textFile("/kaggle/input/kddcup/kddcup.data")

In [ ]:
# cache repartitions
rdd.getNumPartitions()

In [ ]:
spark.sparkContext.defaultParallelism

In [ ]:
# Q1: get ten records randomly
rdd.takeSample(False, 10, 1234)

In [ ]:
rdd.take(2)

In [ ]:
# Q2: rdd count elements
rdd.count()

In [ ]:
# Q3: Calculate the ratio of "normal" connections
normal_rdd = rdd.filter(lambda x: "normal." in x)

In [ ]:
normal_rdd.count()

In [ ]:
ratio = normal_rdd.count()/rdd.count()

In [ ]:
print(f"Ratio of normal connection: {format(round(ratio, 4) * 100)} %")

In [ ]:
# Q4: Get the list of labels
list_of_labels = rdd.map(lambda x: x.split(',')[-1]).distinct().collect()
list_of_labels

In [ ]:
# Q5: Count the number of connections for each label
%time
conn_num_labels = rdd.map(lambda x: (x.split(',')[-1],1)).reduceByKey(lambda x,y: x+y).collect()
conn_num_labels

In [ ]:
# Q6: get the connection type with successfull 'root_shell' connections to servers, 
# where the number of data bytes from source(src_bytes) is 500 times more than from server ()